# Extensions for FluentAssertions


## EventualAssertions

The extension allows to loop FluentAssertions checks for a specified period of time. The extensions relies heavily on the [Assertion Scopes](https://fluentassertions.com/introduction#assertion-scopes) feature and is an assertion scope itself. All assertions wrapped with the extension will be batched and throw an exception only if the timeout has expired. Otherwise, a new attempt will be made for the whole batch.

A particular goal when designing the extension was to provide the experience similar to given by Assertion Scopes, that is, no additional nesting would be needed. This is elegantly achievable in F&sharp;, but a little bit of a challenge for C&sharp;

### EventualAssertions in C#

In [ ]:
#r "nuget: mazharenko.FluentAssertions.Extensions, *"

Installed Packages mazharenko.FluentAssertions.Extensions, 1.0.0

In [ ]:
using mazharenko.FluentAssertions.Extensions;
using FluentAssertions;
using FluentAssertions.Extensions;

var attempts = EventualAssertions.Attempts(4.Seconds(), 400.Milliseconds());

`attempts` now is of a type that implements `IEnumerable` returning `IEnumerator` wrapping an assertion scope. On each new element requested the assertion scope is checked and the specified delay is waited. Here, assertions are supposed to be placed in the body of the loop over `attempts`. Elements of `attempts` can provide some meta information about the attempt.

In [ ]:
var inFuture = DateTime.Now.AddSeconds(3);

foreach (var attempt in attempts)
{
    Console.WriteLine(attempt.Number);
    DateTime.Now.Should().BeAfter(inFuture);
}

1
2
3
4
5
6
7
8
9


In [ ]:
foreach (var _ in EventualAssertions.Attempts(400, 50))
    (1+1).Should().Be(3);

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.

   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message) in /_/Src/FluentAssertions/Execution/FallbackTestFramework.cs:line 21
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in /_/Src/FluentAssertions/Execution/TestFrameworkProvider.cs:line 34
   at FluentAssertions.Execution.CollectingAssertionStrategy.ThrowIfAny(IDictionary`2 context) in /_/Src/FluentAssertions/Execution/CollectingAssertionStrategy.cs:line 42
   at mazharenko.FluentAssertions.Extensions.Eventual.AttemptsEnumerator.Dispose()
   at Submission#4.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

EventualAssertions can be nested in other assertion scopes.

In [ ]:
using FluentAssertions.Execution;

using (new AssertionScope())
{
    foreach (var _ in EventualAssertions.Attempts(400, 50))
    {
        (1+1).Should().Be(3);
        break;
    }
    foreach (var _ in EventualAssertions.Attempts(400, 50))
    {
        (2*2).Should().Be(5);
        break;
    }
}

Error: FluentAssertions.Execution.AssertionFailedException: Expected value to be 3, but found 2.
Expected value to be 5, but found 4.

   at FluentAssertions.Execution.FallbackTestFramework.Throw(String message) in /_/Src/FluentAssertions/Execution/FallbackTestFramework.cs:line 21
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in /_/Src/FluentAssertions/Execution/TestFrameworkProvider.cs:line 34
   at FluentAssertions.Execution.CollectingAssertionStrategy.ThrowIfAny(IDictionary`2 context) in /_/Src/FluentAssertions/Execution/CollectingAssertionStrategy.cs:line 42
   at Submission#5.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

Asynchronous methods are also available. Those would use `Task.Delay` instead of `Thread.Sleep`.

In [ ]:
var inFuture = DateTime.Now.AddSeconds(3);
await foreach(var _ in EventualAssertions.AttemptsAsync(4.Seconds(), 400.Milliseconds()))
{
    DateTime.Now.Should().BeAfter(inFuture);
}